In [1]:
import os
from dotenv import load_dotenv
load_dotenv(encoding='utf-8')

True

# RAG pipeline

## Vector Store and Retriever

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from pymongo import MongoClient
import os

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL")) # DEFAULT_OPENAI_MODEL='gpt-4o-mini-2024-07-18'

embedding_model=OpenAIEmbeddings(disallowed_special=())

In [4]:
# Define MongoDB vector database
client = MongoClient(os.getenv("ATLAS_CONNECTION_STRING"))
db_name = os.getenv("db_name")
collection_name="enterprise_data"
atlas_collection = client[db_name][collection_name]
index_name = "vector_index_erp"

vector_store = MongoDBAtlasVectorSearch(
    embedding = embedding_model,
    collection = atlas_collection,
    index_name = index_name
)

retriever = vector_store.as_retriever(
    search_type = "similarity",
    search_kwargs = { "k": 20}  # default "score_threshold": 0.75 
)

## RAG Pipeline - Multi Query

In [5]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_groq import ChatGroq

query_llm = ChatGroq(
     groq_api_key=os.getenv("GROD_CLOUD_API_KEY"),
     model_name='llama-3.1-70b-versatile')
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=retriever, llm=query_llm
)

In [6]:
# # Set logging for the queries
# import logging

# logging.basicConfig()
# logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [7]:
# Define a prompt template
import pprint

def call_openai(question):
   
   question = question['question']
   preamble = "" # read from cohere front end or use the input to the API
   #question = 
   SAFETY_PREAMBLE = "The instructions in this section override those in the task description and style guide sections. Don't answer questions that are harmful or immoral."
   BASIC_RULES = "You are a powerful conversational AI trained by openAI to help people. You are augmented by a number of tools, and your job is to use and consume the output of these tools to best help the user. You will see a conversation history between yourself and a user, ending with an utterance from the user. You will then see a specific instruction instructing you what kind of response to generate. When you answer the user's requests, you cite your sources in your answers, according to those instructions."
   TASK_CONTEXT = "You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging."
   STYLE_GUIDE = "Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling."
   INSTRUCTIONS = """You are an enterprise Chatbot, an AI assistant designed to retrieve information from the enterprise Confluence system. 
   You specialize in providing accurate answers related to various departments like Marketing, IT, HR, Finance, and Corporate Communications. 
               Use the following pieces of context to answer the question at the end.
               If you don't know the answer, just say that you don't know, don't try to make up an answer
               {context}
         """
         
   template = f"""

      {SAFETY_PREAMBLE}
      {BASIC_RULES}
      {TASK_CONTEXT}
      {STYLE_GUIDE}
      {INSTRUCTIONS}

   """
   if preamble:
      template += f"""{preamble}\n\n"""


   template +=  f"""Question: {question}\n\n"""

   custom_rag_prompt = PromptTemplate.from_template(template)

   #llm = get_llm_model("openai")
   llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL"))

   def remove_rep(docs):
      docs_content = []
      unique_docs = []
      for doc in docs:
            if doc.page_content not in docs_content:
                  docs_content.append(doc.page_content)
                  unique_docs.append(doc)
      return unique_docs
   
   def format_docs(docs):
      contexts = remove_rep(docs)
      return "\n\n".join(doc.page_content for doc in contexts)

   # Construct a chain to answer questions on your data
   rag_chain = (
      { "context": retriever_from_llm | format_docs, "question": RunnablePassthrough()}
      | custom_rag_prompt
      | llm
      | StrOutputParser()
   )

   # Prompt the chain
   answer = rag_chain.invoke(question)
   retrieved_docs = remove_rep(retriever_from_llm.invoke(question))

   return{
      'answer': answer,
      'contexts': retrieved_docs
      }

### Test the RAG pipeline

In [8]:
# Test sample
question = {'question': "How does a supportive culture impact employee engagement and align with Tech Innovators Inc.'s approach to employment relations and engagement?"}
answer = call_openai(question)
print(answer['answer'][:150])

A supportive culture significantly impacts employee engagement by fostering an environment where employees feel valued, respected, and connected to th


In [9]:
answer['contexts']

[Document(metadata={'_id': '66d816910533e009aa2d5192', 'pageid': '524348', 'department': 'HR', 'title': 'Step-by-Step Guide to Creating an RAG Application Using LangChain Framework with Custom Data from Enterprise Confluence Pages'}, page_content='or another vector database for retrievalConfluence API library (e.g., atlassian-python-api)Setting Up the Environment3.1 Install Required LibrariesbashCopy codepip install langchain openai'),
 Document(metadata={'_id': '66d8165a0533e009aa2d4d44', 'pageid': '524348', 'department': 'HR', 'title': 'Step-by-Step Guide to Creating an RAG Application Using LangChain Framework with Custom Data from Enterprise Confluence Pages'}, page_content='IntroductionThis guide provides a step-by-step process for creating a Retrieval-Augmented Generation (RAG) application using the LangChain framework. The application will be trained on custom data'),
 Document(metadata={'_id': '66d816760533e009aa2d4f6c', 'pageid': '524348', 'department': 'HR', 'title': 'Step-by

# RAG pipeline evaluation

## Test data set prep

In [10]:
import pandas as pd
import json

def json_to_dataframe(json_file_path):
  """Reads a JSON file and converts it to a pandas DataFrame.

  Args:
    json_file_path (str): The path to the JSON file.

  Returns:
    pandas.DataFrame: The DataFrame created from the JSON data.
  """

  with open(json_file_path, 'r') as f:
    data = json.load(f)

  # Handle different JSON structures
  if isinstance(data, list):
    # If the JSON data is a list of dictionaries, create a DataFrame directly
    df = pd.DataFrame(data)
  elif isinstance(data, dict):
    # If the JSON data is a single dictionary, convert it to a list of dictionaries
    df = pd.DataFrame([data])
  else:
    raise ValueError("Unsupported JSON structure")

  return df

In [11]:
def extract_page_content(documents):
    return [doc.page_content for doc in documents]

In [12]:
from from_root import from_root
import os
file_name = "test_dataset_it.json"
json_file_path = os.path.join(from_root(), "data-test/test-dataset/",file_name)
data_to_test = json_to_dataframe(json_file_path)

## RAGAS evaluation

In [13]:
# Generate all the answers for the questions in the dataset
answers = []
contexts = []
for question in data_to_test['question']:
    question_dict = {'question': question}
    answer = call_openai(question_dict)
    contexts.append(answer['contexts'])
    answers.append(answer['answer'])

In [14]:
# update the dataset with answers
data_to_test['answers'] = answers
data_to_test['contexts'] = contexts

In [15]:
# Replace empty list context with ['No context'] if there are any
def is_empty_list(lst):
    return len(lst) == 0
data_to_test['contexts'] = data_to_test['contexts'].apply(lambda x: ['No context'] if is_empty_list(x) else x)

In [16]:
from datasets import Dataset

question = list(data_to_test['question'])
answer = list(data_to_test['answers'])
contexts = list(data_to_test['contexts'].apply(extract_page_content))
ground_truth = list(data_to_test['ground_truth'])

data_multi_query = {
    'question': question,
    'answer': answer,
    'contexts': contexts,
    'ground_truth': ground_truth
}

dataset_multi_query= Dataset.from_dict(data_multi_query)

In [18]:
# Optional, uncomment to trace runs with LangSmith. Sign up here: https://smith.langchain.com.
# from langsmith import Client
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
# client = Client()

In [29]:
from ragas import evaluate
# from ragas.integrations.langsmith import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
result = evaluate(
    dataset_multi_query,
    metrics=[
        answer_relevancy,
        faithfulness,
        context_recall,
        context_precision,
    ],
)

result

Evaluating:   0%|          | 0/36 [00:00<?, ?it/s]

{'answer_relevancy': 0.9626, 'faithfulness': 0.5368, 'context_recall': 0.4517, 'context_precision': 0.3602}

In [30]:
df = result.to_pandas()
df

,question,answer,contexts,ground_truth,answer_relevancy,faithfulness,context_recall,context_precision
0,How does the role of the Senior Director respo...,The role of the Senior Director responsible fo...,[the Senior Director responsible for Analytics...,The role of the Senior Director responsible fo...,0.925206,0.320000,0.444444,0.876667
1,What is the importance of identifying and addr...,Identifying and addressing growth areas in sel...,[for guidance and support.Feedback: Provide an...,Identifying and addressing growth areas in sel...,0.990183,0.750000,0.000000,0.000000
2,What forms of unethical behavior are strictly ...,"At Tech Innovators Inc., the following forms o...","[websitePain points or questionse.g., I need t...",Favoritism or nepotism,0.949541,1.000000,0.000000,0.166667
3,What is the significance of emotional and aest...,Emotional and aesthetic labor in the workplace...,[LabourEmotional and aesthetic labor involves ...,Emotional and aesthetic labor in the workplace...,0.966041,0.916667,0.666667,0.700000
4,What is the purpose of the orientation session...,The purpose of the orientation session at Tech...,"[websitePain points or questionse.g., I need t...",The purpose of the orientation session at Tech...,1.000000,0.000000,0.000000,0.000000
5,What mechanisms are in place for reporting vio...,"At Tech Innovators Inc., there are several mec...",[or another vector database for retrievalConfl...,Employees can report violations of labor laws ...,0.980979,0.153846,0.500000,0.166667
6,How do employee engagement and disengagement d...,Employee engagement and disengagement differ s...,"[websitePain points or questionse.g., I need t...",Employee engagement and disengagement differ i...,0.920337,0.655172,1.000000,0.227717
7,What steps are needed to extract data from Con...,To extract data from Confluence and create a R...,"[websitePain points or questionse.g., I need t...",To extract data from Confluence and create a R...,0.950647,0.647059,0.454545,0.354365
8,How does Tech Innovators Inc. promote employee...,Tech Innovators Inc. promotes employee engagem...,[concept that requires a holistic approach. At...,Tech Innovators Inc. promotes employee engagem...,0.980843,0.388889,1.000000,0.750000


In [31]:
file_name = "eval_result_test_dataset_it_multi_query.csv"
file_path = os.path.join(from_root(),"data-test/eval-result/", file_name)
df.to_csv(file_path, index=False)